# Bach Functionality Demonstration

On this page we present a range of mostly synthetic models, set up to highlight individual
functionalities of Bach. The demonstrations are divided across two sections, in @sec-free
free-flowing basins are used, which can drain freely to a downstream basin. In @sec-level
level controlled basins are used. Here there is no free drainage, but water levels are
controlled by water management, such as pumping. Note that this is the only difference
between both, so all other functionality equally applies to free draining and level
controlled basins.

# Demonstration of Free Flowing LSW {#sec-free}

The following examples demonstrate the impact of forcings and the user allocation functionality for a single free draining LSW. The examples are simulated with synthetic data to highlight the functionality of Bach, with the exception of example @sec-free-balance and @sec-free-comparison, which show the Bach prototype simulation of the Hupsel LSW.

In [ ]:
using Bach
using Duet
using Dates
using TOML
using Arrow
using DataFrames
import BasicModelInterface as BMI
using SciMLBase
using Graphs
using CairoMakie
using DataFrameMacros

In [ ]:
# Load LSW Forcings Data
lswforcing = DataFrame(Arrow.Table("../data/input/6/forcing.arrow"));

## No external forcing {#sec-free-no}

This fictional free flowing LSW has no input or output forcing flux. The LSW empties according to the Volume-Area-Discharge relationship.
The two examples below show the impact the profile has on the LSW drainage.

In [ ]:
case = "emptying_sloping_profile"

## Set up
config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
config["starttime"] = Date("2022-01-01")
config["endtime"] = Date("2022-02-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN, depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=DateTime[], variable=Symbol[], location=Int[], value=Float64[])
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot results
Duet.plot_series(reg, lsw_id)

In [ ]:
case = "emptying_steep_profile_Hupsel"

## Set up
config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
config["starttime"] = Date("2022-01-01")
config["endtime"] = Date("2022-02-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN, depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=DateTime[], variable=Symbol[], location=Int[], value=Float64[])
# profile from Hupsel
config["profile"] = DataFrame(location =[1,1,1,1], volume=[0.0, 7427.697265625, 14855.39453125, 750197.375], area=[92152.2890625, 92152.2890625, 92152.2890625, 92152.2890625], discharge=[0.0, 0.0, 0.09600285440683365, 9.600285530090332], level= [18.909799575805664, 19.109800338745117, 19.309799194335938, 39.109798431396484])

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot results
Duet.plot_series(reg, lsw_id)

## Precipitation forcing

This fictional free flowing LSW is simulated with only the external forcing of synthetic precipitation data. The storage and the outflow respond to the preciptation as shown below.

In [ ]:
case = "precipitation"

## Set up
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2020-01-01")
dummydata = @subset(lswforcing, :variable == Symbol("precipitation"), :location == 151358, :time >= config["starttime"], :time <= config["endtime"])
dummydata.location .= lsw_id
config["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=dummydata.location, value=dummydata.value * 3) # Exaggerated for demo
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

# Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

# Plot results
Duet.plot_series(reg, lsw_id)

## Evaporation forcing

The LSW loses water from evaporation. Outflow occurs according to the relation as in @sec-free-no, but overall volume decline is faster rate due to additional loss from evaporation. Evaporation does not occur in an empty LSW.

The second example shows the LSW with no simulated discharge, so that the only output is evaporation

In [ ]:
case = "evaporation"

## Set up
lsw_id = 1
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-06-01")
dummydata = @subset(lswforcing, :variable == Symbol("evaporation"), :location == 151358)
config["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value * 3)
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot results
Duet.plot_series(reg, lsw_id)

In [ ]:
case = "evaporation2"

## Set up
config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
starttime = DateTime("2019-01-01")
config["starttime"] = DateTime("2019-01-01")
config["endtime"] = Date("2019-06-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN,
    depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=starttime, variable=:evaporation, location=lsw_id,
    value=1e-6)
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 0.0], level=[10.0, 11.0])

# Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

# Plot
Duet.plot_series(reg, lsw_id)

## Evaporation and precipitation forcings

This example shows the evaporation and precipitation flux simulated together.

In [ ]:
case = "evaporation_precipitation"

## Set up
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-06-01")
dummydata_e = @subset(lswforcing, :variable == Symbol("evaporation"), :location == 151358)
dummydata_p = @subset(lswforcing, :variable == Symbol("precipitation"), :location == 151358)
dummydata = append!(dummydata_e, dummydata_p);
sort!(dummydata, [order(:time, rev=false)]);
config["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value * 3)

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot
Duet.plot_series(reg, lsw_id)

## Infiltration 

Infiltration is a negative flux of the LSW. This example shows the LSW storage responding to an enhanced forcing of infiltration. There is no outflow simulated of this LSW, so that the only output is the infiltration.

In [ ]:
case = "Infiltration"

## Set up
config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
starttime = DateTime("2019-01-01")
config["starttime"] = starttime
config["endtime"] = Date("2019-02-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN,
    depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=starttime, variable=:infiltration,
    location=lsw_id, value=1.5e-6)
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 0.0], level=[10.0, 11.0])

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot results
Duet.plot_series(reg, lsw_id)

## Urban Runoff

Urban runoff is a surface water input to the LSW. In case of the Netherlands, this is not calculated by MODFLOW but by the unsaturated zone model MetaSWAP.
This example shows the LSW storage responding to the influx forcing of urban runoff.

In [ ]:
case = "Urban Runoff"

## Set up
config["starttime"] = Date("2019-01-01")
config["endtime"] = Date("2019-06-01")
dummydata = @subset(lswforcing, :variable == Symbol("urban_runoff"), :location == 151358)
dummydata.value *= 100 # Emphasised to highlight the functionality
config["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value)

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot
Duet.plot_series(reg, lsw_id)

## Allocation to multiple users (agriculture and industry)

The allocation is based upon demand and prioritisation of the users and the available water in the LSW. In a free flowing LSW only water from the LSW can be abstracted by the users: agriculture and industry.
In this example there are two users. Agriculture has higher prioirty than industry, therefore when there is a shortage of available water, agriculture abstracts water before industry as demonstrated.

When water supply is limited, the model follows “de verdringingsreeks” (water prioritization rules in times of water shortage in the Netherlands).

<img src="https://user-images.githubusercontent.com/103200724/187085506-0ee79dd0-8023-42e8-9b05-4b9f433ae73e.jpg" alt="Multi user" title="Bach multi user allocation" width="800"/>

## Water balance of a single LSW (Hupsel) {#sec-free-balance}

This simulation is for the LSW Hupsel. The LSW is a free flowing LSW.

<img src="https://user-images.githubusercontent.com/103200724/187200598-edae91ad-8140-466a-b06e-3289babeee27.png" alt="Hupsel" title="Hupsel Water Balance" width="800"/>

## Water balance comparison Hupsel {#sec-free-comparison}

The following two figures show Hupsel LSW water balance for the Bach prototype compared to the water balance simulated by Mozart, the precursor to Bach.
The figures show a good agreement between the two simulations.

<img src="https://user-images.githubusercontent.com/4471859/179259174-0caccd4a-c51b-449e-873c-17d48cfc8870.png" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892581-557e2a0a-7b17-47cc-b65c-a2f7e3e30b66.PNG" alt="Mozart - Bach Storage Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892581-557e2a0a-7b17-47cc-b65c-a2f7e3e30b66.PNG" alt="" title="Mozart - Bach Storage Comparison" width="800"/>

## Bifurcation

To handle bifurcations, we want to be able to model them using a fraction. To implement this
we use a separate `Bifurcation` node, that has an inflow `a`, and outflows `b` and `c`.
The fraction `f` of the inflow coming into `a` will go to `b`, the rest will go to `c`.
These fractions are defined for every link in the network. Only for nodes that have multiple
downstream nodes, will they not be one. When constructing a network with such a bifurcation,
the bifurcation nodes are automatically inserted. Below we show the result of a small
simulation on three LSW IDs. From ID 151380 50% flows to 151392 and 50% to 151318.

In [ ]:
case = "Bifurcation"

## Set up
config = Dict{String,Any}()
lsw_ids = [151380,151392,151318]
config["lsw_ids"] = lsw_ids
config["update_timestep"] = 86400.0
config["starttime"] = DateTime("2022-01-01")
config["endtime"] = DateTime("2022-02-01")
config["state"] = DataFrame(location=lsw_ids, volume=[1e6,0.0,0.0])
config["static"] = DataFrame(location=lsw_ids, target_level=NaN, target_volume=NaN, depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=DateTime[], variable=Symbol[], location=Int[], value=Float64[])
config["profile"] = DataFrame(location=[151318,151318,151380,151380,151392,151392], volume=[0.0, 1e6, 0.0, 1e6, 0.0, 1e6], area=1e6, discharge=[0.0, 1e0, 0.0, 1e0, 0.0, 1e0], level=[10.0, 11.0, 10.0, 11.0, 10.0, 11.0])
config["network_path"] = "../data/input/6/network.ply"

## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)

## Plot results
fig = Figure()
ylabel = "flow rate / m³ s⁻¹"
timespan = datetime2unix(config["starttime"])..datetime2unix(config["endtime"])
ax = Duet.time!(Axis(fig[1, 1]; ylabel), timespan.left, timespan.right)

name = Symbol(:bifurcation_, 151380)
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊a₊Q)), label = "bifurcation inflow")
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊b₊Q)), label = "bifurcation outflow 1")
lines!(ax, timespan, interpolator(reg, Symbol(name, :₊c₊Q)), label = "bifurcation outflow 2", linestyle=:dashdot)
fig[1, 2] = Legend(fig, ax, "", framevisible = true)
hidexdecorations!(ax, grid = false)
fig

# Demonstration of a Level Controlled LSW {#sec-level}

The following examples demonstrate the water management and user allocation functionality for a level controlled LSW. Polder de Tol is used as the example for these demonstrations.

## Single Level Controlled LSW (Tol)

This simulation is for the LSW De Tol. The LSW is a level controlled meaning that water is allocated to maintain water levels at a target level. 

<img src="https://user-images.githubusercontent.com/103200724/187208026-22b39c44-03c4-4694-8f82-79a0fa52551e.png" alt="tol" title="Tol Water Balance" width="800"/>




## De Tol water balance comparison with Mozart

The following two figures show De Tol LSW water balance for the Bach prototype compared to the water balance simulated by Mozart, the precursor to Bach.
The figures show a good agreement between the two simulations

<img src="https://user-images.githubusercontent.com/103200724/186892605-db217b55-13d6-48bb-a78e-3bbf4acc70c1.PNG" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892628-7cb0d3cd-9439-46cb-b6b8-9908d9c7e09a.PNG" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>